In [1]:
using BenchmarkTools

In [3]:
include("Main.jl")
AN=AtomNum(82,125,1)
rmesh=getrmesh()
@btime InitPot(AN,rmesh)

isGuleria=0
  18.000 μs (16 allocations: 30.52 KiB)


([20.74981110764005 20.74981110764005 … 20.74981110764005 20.74981110764005; 20.721248539064334 20.721248539064334 … 20.721248539064334 20.721248539064334; 17.450269114874917 17.450269114874917 … 17.450269114874917 17.450269114874917], [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], [-57.85429862919295 -57.854019835306005 … -7.226472995447973e-14 -5.312480057329455e-14; -45.14432370489379 -45.14410615906268 … -5.638893632470049e-14 -4.145384613873059e-14; -34.666202986229834 -34.666035933379746 … -4.3300910333445764e-14 -3.183229533349714e-14], [-0.0004092339650531184 -0.0005566683852758072 … -3.82169244951575e-14 -2.8094846457030748e-14; -0.00031932960950412423 -0.00043437425354061614 … -2.982107209479349e-14 -2.192270709259789e-14; 0.0 0.0 … 0.0 0.0])

In [5]:
include("Main.jl")
AN=AtomNum(82,125,1)
@code_warntype(InitialCondition(AN))

isGuleria=0
MethodInstance for InitialCondition(::AtomNum)
  from InitialCondition(AN::AtomNum) in Main at c:\Users\jinno\OneDrive\progmrams\Guleria2012\Main.jl:296
Arguments
  #self#::Core.Const(InitialCondition)
  AN::AtomNum
Locals
  @_3::Int64
  InitState::Vector{Vector{SingleParticleState}}
  W::Matrix{Float64}
  V::Matrix{Float64}
  ddh2m::Matrix{Float64}
  dh2m::Matrix{Float64}
  h2m::Matrix{Float64}
  rmesh::StepRangeLen{Float64, Base.TwicePrecision{Float64}, Base.TwicePrecision{Float64}, Int64}
Body::Vector{Vector{SingleParticleState}}
1 ─       (rmesh = Main.getrmesh())
│   %2  = Main.InitPot(AN, rmesh)::NTuple{5, Matrix{Float64}}
│   %3  = Base.indexed_iterate(%2, 1)::Core.PartialStruct(Tuple{Matrix{Float64}, Int64}, Any[Matrix{Float64}, Core.Const(2)])
│         (h2m = Core.getfield(%3, 1))
│         (@_3 = Core.getfield(%3, 2))
│   %6  = Base.indexed_iterate(%2, 2, @_3::Core.Const(2))::Core.PartialStruct(Tuple{Matrix{Float64}, Int64}, Any[Matrix{Float64}, Core.Const(3)])
│

In [12]:
#Allocation 4826029. Huge!
#Allocation 3272257, 99.482 ms (Change Rin[1:4]=Rin[2:5]-> for loop.)
#Allocation 3272257, 100.600 ms (Change Rout[1,:],Rout[2],Rout[3])
#Allocation 3271901, 97.989 ms (Change R[Nmatch:Nmesh]/=R[Nmatch])
include("Main.jl")
function BenchInitialCondition()
	AN=AtomNum(82,125,1)
	@btime (InitialCondition(AN))
	println()
end
BenchInitialCondition()

isGuleria=0
  97.989 ms (3271901 allocations: 248.63 MiB)



In [37]:
#Whats the bottleneck of the InitialCondition?
#RadWaveFunc is fast enough!
include("Main.jl")
function BenchRadWaveFunc()
	rmesh=getrmesh()
	AN=AtomNum(82,125,1)
    h2m,dh2m,ddh2m,V,W=InitPot(AN,rmesh)
	b=1
	QN=QuantumNumber(0.5,0,b)
	A,C=CalcABC(QN,h2m[b,:],dh2m[b,:],ddh2m[b,:],V[b,:],W[b,:],rmesh)
	@time RadWaveFunc(-40,QN,A,C,rmesh)
	println()
end
BenchRadWaveFunc()

isGuleria=0
  0.000029 seconds (304 allocations: 33.062 KiB)



In [15]:
#0.002314 seconds (81.80 k allocations: 6.228 MiB)
#MyBisect is the Bottleneck.
include("Main.jl")
function BenchCalcStates()
	rmesh=getrmesh()
	AN=AtomNum(82,125,1)
    h2m,dh2m,ddh2m,V,W=InitPot(AN,rmesh)
	b=1
	QN=QuantumNumber(0.5,0,b)
	@time CalcStates(QN,h2m[b,:],dh2m[b,:],ddh2m[b,:],V[b,:],W[b,:],rmesh)
	@time CalcStates(QN,h2m[b,:],dh2m[b,:],ddh2m[b,:],V[b,:],W[b,:],rmesh)
	@time CalcStates(QN,h2m[b,:],dh2m[b,:],ddh2m[b,:],V[b,:],W[b,:],rmesh)
	println()
end
BenchCalcStates()

isGuleria=0
  0.123146 seconds (290.82 k allocations: 15.862 MiB, 99.95% compilation time)
  0.000036 seconds (622 allocations: 48.031 KiB)
  0.000022 seconds (622 allocations: 48.031 KiB)
  0.000018 seconds (622 allocations: 48.031 KiB)
  0.000034 seconds (622 allocations: 48.031 KiB)
  0.000034 seconds (622 allocations: 48.031 KiB)
  0.000019 seconds (622 allocations: 48.031 KiB)
  0.000018 seconds (622 allocations: 48.031 KiB)
  0.000017 seconds (622 allocations: 48.031 KiB)
  0.000299 seconds (5.60 k allocations: 432.266 KiB)
  0.000035 seconds (622 allocations: 48.031 KiB)
  0.000202 seconds (5.60 k allocations: 432.266 KiB)
  0.000049 seconds (622 allocations: 48.031 KiB)
  0.000027 seconds (622 allocations: 48.031 KiB)
  0.000253 seconds (5.91 k allocations: 456.281 KiB)
  0.000018 seconds (622 allocations: 48.031 KiB)
  0.000021 seconds (622 allocations: 48.031 KiB)
  0.000021 seconds (622 allocations: 48.031 KiB)
  0.000201 seconds (6.53 k allocations: 504.312 KiB)
  0.000024 

In [14]:
#3.27 M allocations: 0.1272565 sec. There is a bottle neck!
include("Main.jl")
function BenchCalcAllStates()
	rmesh=getrmesh()
	AN=AtomNum(82,125,1)
    h2m,dh2m,ddh2m,V,W=InitPot(AN,rmesh)
	b=1
	@time CalcAllStates(h2m,dh2m,ddh2m,V,W,rmesh)
	@time CalcAllStates(h2m,dh2m,ddh2m,V,W,rmesh)
	@time CalcAllStates(h2m,dh2m,ddh2m,V,W,rmesh)
	println()
end
BenchCalcAllStates()

isGuleria=0
  0.000002 seconds (2 allocations: 208 bytes)
  0.000002 seconds (2 allocations: 176 bytes)
  0.000002 seconds (2 allocations: 160 bytes)
  0.255194 seconds (1.77 M allocations: 120.527 MiB, 10.04% gc time, 80.78% compilation time)
  0.000003 seconds (2 allocations: 208 bytes)
  0.000002 seconds (2 allocations: 176 bytes)
  0.000002 seconds (2 allocations: 160 bytes)
  0.037898 seconds (1.10 M allocations: 84.646 MiB, 14.01% gc time)
  0.000002 seconds (2 allocations: 208 bytes)
  0.000003 seconds (2 allocations: 176 bytes)
  0.000001 seconds (2 allocations: 160 bytes)
  0.035137 seconds (1.10 M allocations: 84.646 MiB, 12.49% gc time)

